# Snake species detection using transfer learning

### Importing the modules required

We will be using all these packages for this notebook, to install any missing packages you can use `python-pip`

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
import time
import os
import copy
from shutil import copyfile
from os import listdir
from PIL import Image
import csv
plt.ion()   # interactive mode

### Splitting training data

Here we split our training data into training and validation set. The current validation set is 20% of the whole data. You can vary it using the `train_fraction` variable below

In [ ]:
directories = ["data/train_data", "data/val_data"] 

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        
for i in os.listdir("data/train"):
    if not os.path.exists(os.path.join(directories[0], i)):
        os.makedirs(os.path.join(directories[0], i))
    if not os.path.exists(os.path.join(directories[1], i)):
        os.makedirs(os.path.join(directories[1], i))    
        
train_fraction = 0.8

for i in os.listdir("data/train"):
    images = os.listdir(os.path.join("data/train/", i))
    n_images = len(images)
    n_train_images = int(n_images*train_fraction)
    n_val_images = n_images - n_train_images

    for j in range(n_train_images):
        copyfile(os.path.join(os.path.join("data/train/", i), images[j]) , os.path.join(os.path.join("data/train_data/", i), images[j]))
        
    for j in range(n_val_images):
        copyfile(os.path.join(os.path.join("data/train/", i), images[n_train_images + j]) , os.path.join(os.path.join("data/val_data/", i), images[n_train_images + j]))
    
for root in ['data/train_data', 'data/val_data']:
    for i in listdir(root):
        for j in listdir(os.path.join(root, i)):
            try:
                img = Image.open(os.path.join(os.path.join(root, i), j)) # open the image file
                img.verify() # verify that it is, in fact an image
            except (IOError, SyntaxError) as e:
                if os.path.exists(os.path.join(os.path.join(root, i), j)):
                    os.remove(os.path.join(os.path.join(root, i), j))

### Load Data

Here we transform our images so that we can feed them to a pretrained machine learning model (ResNet).

We will use torchvision and torch.utils.data packages for loading the data.

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train_data': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_data': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train_data', 'val_data']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train_data', 'val_data']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train_data', 'val_data']}
class_names = image_datasets['train_data'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
with open('data/class_idx.csv', newline='') as csv_file:
    data = list(csv.reader(csv_file))
    
class_idx= {}
for line in data[1:]:
    class_idx[line[1]] = line[0]

### Visualization

Here we view some of the images from the dataset which we will be feeding to our machine learning model.

In [ ]:
plt.figure(figsize=(10,8))
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train_data']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_idx[class_names[x][class_names[x].rfind('-')+1:]] for x in classes])

### Train model

The next step would be to train a pretrained model on our dataset to fine tune the model.

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train_data', 'val_data']:
            if phase == 'train_data':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train_data'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train_data':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Visualizing specifications of our model

Once trained we will visualize our model using this method

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val_data']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

### Configuring model

We have 45 unique classes and we will be using resnet18 with pretrained weights, we will be adding a fully connected layer at the end to fine tune the pretrained model. This way of training a machine learning model is known as transfer learning.

We will be using categorical crossentropy as our loss function which is highly suitable for multi category classification.

We will be using basic Stochastic Gradient Descent to optimize our loss with decay of learning rate for better convergence

In [ ]:
num_classes = 45
efficient = False
if efficient:
    model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=num_classes) 
else:
    model = models.resnet18(pretrained=True)
    
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Finally we train our model using the method defined above

In [ ]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=100)

Here we have some predictions by our model which was fine tuned with 5 epochs

In [ ]:
visualize_model(model_ft)

### Summary


1) We split the give data into training and validation data

2) We normalized all the images, so they fit into the model we will be using

3) We use resnet18 with pretrained weights and add a fully connected layer to it. This way we fine tune the model on our data

4) We train our model for 5 epochs and check accuracy on the validation data

5) It was observed that we got an accuracy of 0.64 on our validation data which was never seen by our model